<a href="https://colab.research.google.com/github/yn2a75/benkyokai/blob/main/BERT_ALBERT%E5%9F%BA%E7%A4%8E%E3%81%AE%E6%BC%94%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 以下の文に対して MLM の実装を行う  
  - 使用する文章：「私は、ポチという名前の犬を飼っている。今日、＊と散歩に行った。」  
  - ＊に当てはまる単語を MLM で予測
- HuggingFace Transformers は以下の要件を満たして使用する
  - Transformers のバージョン：4.0.0
  - 使用するクラス：BertForMaskedLM
  - 使用するモデルのチェックポイント ：cl-tohoku/bert-base-japanese

【参考】   
https://tech.retrieva.jp/entry/2020/11/27/111202

In [1]:
#環境構築、必要なライブラリのインストール
! pip install -q transformers==4.4.0
! pip install -q sentencepiece
! pip install -q fugashi
! pip install -q ipadic

In [2]:
#必要なモジュールのインポート
import torch
import transformers

transformers.__version__

'4.4.0'

In [3]:
#Bertモデルとtokenizerの準備
from transformers import BertForMaskedLM, BertJapaneseTokenizer


In [4]:
#分かち書き用のtokenizer
bert_tokenizer=BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

In [5]:
#学習済みのBertMaskedeLMモデル
bert=BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
#インスタンス化したBertモデルの可視化
print(bert)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [7]:
text='私は、ポチという名前の犬を飼っている。今日、[MASK]と散歩に行った。'

In [8]:
input_ids1=bert_tokenizer.encode(text,return_tensors='pt')

In [9]:
print(input_ids1)

tensor([[    2,  1325,     9,     6,   329, 28555,   140,  1381,     5,  2928,
            11, 24811,    16,    33,     8,  3246,     6,     4,    13, 19253,
             7,   517,    10,     8,     3]])


In [10]:
print(bert_tokenizer.convert_ids_to_tokens(input_ids1[0]))

['[CLS]', '私', 'は', '、', 'ポ', '##チ', 'という', '名前', 'の', '犬', 'を', '飼っ', 'て', 'いる', '。', '今日', '、', '[MASK]', 'と', '散歩', 'に', '行っ', 'た', '。', '[SEP]']


17番目のtokenに[MASK]が入っている。
このtokenを予測したい。

In [11]:
y=bert(input_ids1)
result=y[0]
print(result.size())

torch.Size([1, 25, 32000])


https://pytorch.org/docs/stable/generated/torch.topk.html


In [12]:
#torch.topk:Returns the k largest elements of the given input tensor along a given dimension.
_, msk_ids =torch.topk(result[0][17],k=15)
result_words=bert_tokenizer.convert_ids_to_tokens(msk_ids.tolist())
print(result_words)

['友達', '家族', '友人', '母親', '夫', '妻', '彼女', 'みんな', '両親', '私', '犬', '母', '父', '仲間', '父親']


#pipelineを使う

In [13]:
from transformers import BertConfig
from transformers import pipeline


In [14]:
from transformers.models.bert.configuration_bert import BertConfig
config=BertConfig.from_pretrained('cl-tohoku/bert-base-japanese')

In [15]:
fill_mask=pipeline('fill-mask',model=bert,tokenizer=bert_tokenizer,config=config)

In [16]:
#fill_mask=bert.fill-mask(input_ids1)

In [17]:
print(fill_mask(text))

[{'sequence': '私 は 、 ポチ という 名前 の 犬 を 飼っ て いる 。 今日 、 友達 と 散歩 に 行っ た 。', 'score': 0.16103681921958923, 'token': 12455, 'token_str': '友 達'}, {'sequence': '私 は 、 ポチ という 名前 の 犬 を 飼っ て いる 。 今日 、 家族 と 散歩 に 行っ た 。', 'score': 0.10103119909763336, 'token': 2283, 'token_str': '家 族'}, {'sequence': '私 は 、 ポチ という 名前 の 犬 を 飼っ て いる 。 今日 、 友人 と 散歩 に 行っ た 。', 'score': 0.09011860936880112, 'token': 3676, 'token_str': '友 人'}, {'sequence': '私 は 、 ポチ という 名前 の 犬 を 飼っ て いる 。 今日 、 母親 と 散歩 に 行っ た 。', 'score': 0.052637457847595215, 'token': 3889, 'token_str': '母 親'}, {'sequence': '私 は 、 ポチ という 名前 の 犬 を 飼っ て いる 。 今日 、 夫 と 散歩 に 行っ た 。', 'score': 0.04743942245841026, 'token': 1532, 'token_str': '夫'}]


In [18]:
y = bert